# ID3

Decision tree is a common machine learning algorithm that is based on tree structure. The goal of decision tree learning is to generate a tree with good generalization.

## Information Gain

Information entropy is the average amount of information conveyed by an event, when considering all possible outcomes. Assume the percentage of the $k$th class is $p_k$ in set $D$, the information entropy of $D$ is defined by,

\begin{equation}
Ent(D) = -\sum_{k=1}^{n}p_{k}\log_{2}p_k
\end{equation}

The smaller the $Ent(D)$, the higher the purity of $D$.
The information gain is defined by,

\begin{equation}
Gain(D,a) = Ent(D)-\sum_{v=1}^{V}\frac{|D^v|}{|D|}Ent(D^v)
\end{equation}

where $a$ is a feature e.g., a column of a Dataframe containing $V$ classes. $D^v$ the number of samples in $k$th class. $\frac{|D^v|}{|D|}$ implies the class with more samples impacts more.
Higher information gain means the information entropy of the feature $a$ is lower - the purity of this attribute is higher, in other words, more entropy is removed.

The ID3 algorithm choose the feature with the highest information gain to build the tree.

In [38]:
import numpy as np
import pandas as pd
import pprint

In [39]:
df = pd.read_csv("Dataset\Watermelon2.0.txt")
df.pop('编号')
df

,色泽,根蒂,敲声,纹理,脐部,触感,好瓜
0,青绿,蜷缩,浊响,清晰,凹陷,硬滑,是
1,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,是
2,乌黑,蜷缩,浊响,清晰,凹陷,硬滑,是
3,青绿,蜷缩,沉闷,清晰,凹陷,硬滑,是
4,浅白,蜷缩,浊响,清晰,凹陷,硬滑,是
5,青绿,稍蜷,浊响,清晰,稍凹,软粘,是
6,乌黑,稍蜷,浊响,稍糊,稍凹,软粘,是
7,乌黑,稍蜷,浊响,清晰,稍凹,硬滑,是
8,乌黑,稍蜷,沉闷,稍糊,稍凹,硬滑,否
9,青绿,硬挺,清脆,清晰,平坦,软粘,否


In [40]:
class ID3:
    def __init__(self):
        self.tree = None

    def info_entropy(self, col):
        entropy = -sum([prob * np.log2(prob) for prob in col.value_counts()/len(col)])
        return entropy

    def split_dataframe(self, df, col_name):
        result_dict = {}

        for key in df[col_name].unique():
            result_dict[key]= df[:][df[col_name]==key]

        return result_dict

    def info_gain(self, df, label):
        Ent = self.info_entropy(df[label])
        cols = [col for col in df.columns if col != label]
        max_gain = float('-inf')
        best_col = None
        best_sub_df = None

        for col in cols:
            sub_df = self.split_dataframe(df, col)
            Ent_A = 0.0

            for value in sub_df.values():
                Ent_i = self.info_entropy(value[label])
                Ent_A += len(value)/len(df)*Ent_i
            gain = Ent-Ent_A

            if gain > max_gain:
                max_gain = gain
                best_col = col

                for key in sub_df.values():
                    key.drop(columns=best_col, inplace=True)
                best_sub_df = sub_df

        return max_gain, best_col, best_sub_df

    def fit(self, data, label):
        self.tree = self._train(data, label)

    def _train(self, data, label, root = None):
        _, best_col, sub_df = self.info_gain(data, label)

        if sub_df is None:
            return

        if root is None:
            root = {}
            root[best_col] = {}

        for key, value in sub_df.items():
            if len(np.unique(value[label])) == 1:
                root[best_col][key] = value[label].iloc[0]
            else:
                root[best_col][key] = self._train(value, label)

        return root

    def predict(self, data):
        return self._predict(data, self.tree)

    def _predict(self, data, root):

        for key in root.keys():
            value = data[key]
            root = root[key][value]
            prediction = None

            if type(root) is dict:
                prediction = self._predict(data, root)

            else:
                prediction = root
                break

        return prediction

label = '好瓜'
tree = ID3()
tree.fit(df, label)
pprint.pprint(tree.tree)

{'纹理': {'模糊': '否',
        '清晰': {'根蒂': {'硬挺': '否',
                      '稍蜷': {'色泽': {'乌黑': {'触感': {'硬滑': '是', '软粘': '否'}},
                                    '青绿': '是'}},
                      '蜷缩': '是'}},
        '稍糊': {'触感': {'硬滑': '否', '软粘': '是'}}}}


## Summery

1. The term $\frac{|D^v|}{|D|}$ makes the ID3 prefer the feature with more classes.
2. A little tricky to build the tree using recursion.
3. ID3 is quite easy to overfit the data, to avoid overfitting, smaller trees should be preferred over larger ones; pruning should be used in practice.